# Connecting to Google Adwords API

In [96]:
from googleads import adwords
from googleads import oauth2
from oauth2client import client
from __future__ import division

import pandas
import zlib

In [2]:
# OAuth 2.0 credential information. In a real application, you'd probably be
# pulling these values from a credential storage.
CLIENT_ID = '845705914022-5ntkratcadgc2i13cic6cl6dg1vkefii.apps.googleusercontent.com'
CLIENT_SECRET = 'W8Ocv49ZWKkAlSP0s81moPxJ'

# AdWords API information.
DEVELOPER_TOKEN = '4dZVjzaMdTxJbiCMoyDKQg'
# Format is <CompanyURL>:<AppName>:<Version>
# The user agent is optional but can help Google Debug if we need their help
# http://googleadsdeveloper.blogspot.ca/2013/11/please-set-user-agent-or-application.html
USER_AGENT = 'quillengage.narrativescience.com:QuillEngage:2.0.0'
CLIENT_CUSTOMER_ID = '319-297-2406' # This is our Narrative Science Quill Engage Account

In [3]:
api_scopes = 'https://www.googleapis.com/auth/adwords'
"""Retrieve and display the access and refresh token."""
flow = client.OAuth2WebServerFlow(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=[api_scopes],
    user_agent='Ads Python Client Library',
    redirect_uri='urn:ietf:wg:oauth:2.0:oob')

authorize_url = flow.step1_get_authorize_url()
print ('Log into the Google Account you use to access your AdWords account'
         'and go to the following URL: \n%s\n' % (authorize_url))

Log into the Google Account you use to access your AdWords accountand go to the following URL: 
https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fadwords&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=845705914022-5ntkratcadgc2i13cic6cl6dg1vkefii.apps.googleusercontent.com&access_type=offline



In [4]:
# PASTE YOUR CODE FROM THE BROWSER HERE
code = '4/COV8mpZdA1T_-5HRPuNMOXxMs-iZZe4p724cGmJpWQg.4lpB-6YNSQUaXmXvfARQvtiY9PhzmwI'

In [5]:
try:
    credential = flow.step2_exchange(code)
except client.FlowExchangeError, e:
    print 'Authentication has failed: %s' % e
else:
    access_token = credential.access_token
    refresh_token = credential.refresh_token

In [6]:
print refresh_token
oauth2_client = oauth2.GoogleRefreshTokenClient(
    CLIENT_ID, CLIENT_SECRET, refresh_token)
adwords_client = adwords.AdWordsClient(
    DEVELOPER_TOKEN, oauth2_client, USER_AGENT, CLIENT_CUSTOMER_ID)

customer = adwords_client.GetService('CustomerService').get()
print 'You are logged in as customer: %s with currency %s at %s in timezone %s' % (
    customer['customerId'], customer['currencyCode'], customer['companyName'], customer['dateTimeZone'])

1/z76hKi8ZmEhIBmNLwxQD6yS74boRYYOqOoTgKwJOdS_BactUREZofsF9C7PrpE-j
You are logged in as customer: 3192972406 with currency USD at Narrative Science in timezone America/Chicago


# Campaign Analysis

In [110]:
# Retrieve the Report Definition of the service we are interested in (Campaign Performance)
fields = ['CAMPAIGN_PERFORMANCE_REPORT']
report_def_service = adwords_client.GetService('ReportDefinitionService')
report_def = report_def_service.getReportFields(fields)

# Define the selector
keywords_selector = {
    'selector': {
        'fields': [
            # Attributes
            'Amount', 'CampaignName', 'CampaignId', 'CampaignStatus', 'TotalBudget', 
            # Metrics
            'AverageCpc', 'Ctr', 'ConversionRateManyPerClick', 'Conversions', 'Cost', 'Impressions',
            'ConversionValue', 'Clicks',
            # Segments
            'Date'
        ],
        'dateRange': {
            'min': '20140401',
            'max': '20140430'
        }
    },
    'reportName': 'Custom Keywords Performance Report',
    'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
    'dateRangeType': 'CUSTOM_DATE',
    'downloadFormat': 'CSV'
}

# BEWARE: Google returns Money in micro format. So $1.23 is 1230000 (1.23 x 1,000,000)
report_downloader = adwords_client.GetReportDownloader()
report = report_downloader.DownloadReportAsStream(keywords_selector)

# Read CSV into Pandas -- Skip row one which is the name of the CSV
# Index on the next column which defines the column names
df = pandas.read_csv(report, skiprows=1, index_col =1)

# And finally discard the final row which has a sum/total
df = df.ix[:-1]
df = df.reset_index()
df = df.sort(['Day'])
df = df.set_index(['Day'], drop=True)
df.index.name = None
campaign_df = df
campaign_df.head()

,Campaign,Budget,Campaign ID,Campaign state,Avg. CPC,CTR,Conv. rate,Converted clicks,Cost,Impressions,Total conv. value,Clicks
2014-04-01,Quill Engage,15000000,166806160,paused,950000,4.76%,0.00%,0,950000,21,0,1
2014-04-01,API,20000000,166805920,paused,1790000,0.77%,0.00%,0,1790000,130,0,1
2014-04-01,Analytics,25000000,166805800,paused,5495000,0.20%,0.00%,0,10990000,988,0,2
2014-04-01,Display - Topics & Interests,1000000,167400880,paused,681875,0.20%,0.00%,0,21820000,16102,0,32
2014-04-01,Display - Content,1000000,167400760,paused,835556,0.18%,3.70%,1,22560000,14719,0,27


In [63]:
campaign_df['Campaign'].unique()

array(['API', 'Quill Engage', 'Reporting', 'Display - Topics & Interests',
       'Display - Content', 'Analytics', 'Remarketing'], dtype=object)

In [111]:
qe_df = campaign_df[campaign_df['Campaign'] == 'Quill Engage']

## Basic payload for campaigns

In [120]:
print qe_df['Campaign state'][-1]
print qe_df['Cost'].sum()
print qe_df['Budget'].astype(int).sum()
print qe_df['Converted clicks'].sum()

paused
19690000
420000000
3


In [119]:
print qe_df['Budget'].astype(int).sum()

420000000


In [51]:
qe_df[['Conv. rate', 'Converted clicks', 'Clicks']]

,Conv. rate,Converted clicks,Clicks
2014-04-01,0.00%,0,1
2014-04-02,0.00%,0,0
2014-04-03,0.00%,0,0
2014-04-04,100.00%,2,2
2014-04-05,0.00%,0,0
2014-04-06,0.00%,0,0
2014-04-07,0.00%,0,1
2014-04-08,0.00%,0,1
2014-04-09,0.00%,0,1
2014-04-10,0.00%,0,1


# Adgroup Analysis

In [101]:
PERFORMANCE_REPORT = 'ADGROUP_PERFORMANCE_REPORT'
ATTRS = ['AdGroupId', 'AdGroupName', 'CampaignId', 'CampaignName', 'CampaignStatus', 'AdGroupStatus']
METRICS = ['AverageCpc', 'Ctr', 'ConversionRateManyPerClick', 'Conversions', 'Cost', 'Impressions', 
           'ConversionValue', 'Clicks', 'AverageCpm']
SEGMENTS = ['Date']

keywords_selector = {
    'selector': {
        'fields': ATTRS + METRICS + SEGMENTS,
        'dateRange': {
            'min': '20140401',
            'max': '20140430'
        }
    },
    'reportName': 'Custom Keywords Performance Report',
    'reportType': PERFORMANCE_REPORT,
    'dateRangeType': 'CUSTOM_DATE',
    'downloadFormat': 'CSV'
}

# BEWARE: Google returns Money in micro format. So $1.23 is 1230000 (1.23 x 1,000,000)
report_downloader = adwords_client.GetReportDownloader()
report = report_downloader.DownloadReportAsStream(keywords_selector)

# Read CSV into Pandas -- Skip row one which is the name of the CSV
# Index on the next column which defines the column names
df = pandas.read_csv(report, skiprows=1, index_col =1)

# And finally discard the final row which has a sum/total
df = df.ix[:-1]
df = df.reset_index()
df = df.sort(['Day'])
df = df.set_index(['Day'], drop=True)
df.index.name = None
adgroup_df = df
adgroup_df.head()

,Ad group,Ad group ID,Campaign ID,Campaign,Campaign state,Ad group state,Avg. CPC,CTR,Conv. rate,Converted clicks,Cost,Impressions,Total conv. value,Clicks,Avg. CPM
2014-04-01,Sales,13789035760,166805800,Analytics,paused,paused,0,0.00%,0.00%,0,0,24,0,0,0
2014-04-01,Business Analytics,13909826440,167400760,Display - Content,paused,enabled,490000,0.10%,0.00%,0,490000,1044,0,1,469349
2014-04-01,Marketing,13909827520,167400880,Display - Topics & Interests,paused,enabled,0,0.00%,0.00%,0,0,77,0,0,0
2014-04-01,Data Analytics,13909826560,167400760,Display - Content,paused,paused,0,0.00%,0.00%,0,0,411,0,0,0
2014-04-01,Custom,13789036840,166806040,Reporting,paused,paused,0,0.00%,0.00%,0,0,34,0,0,0


In [67]:
adgroup_df['Campaign'].unique()

array(['Analytics', 'Reporting', 'Quill Engage', 'API',
       'Display - Topics & Interests', 'Display - Content', 'Remarketing'], dtype=object)

In [124]:
qe_df = adgroup_df[adgroup_df['Campaign'] == 'Quill Engage']

In [125]:
qe_df['Ad group'].unique()

array(['Brand', 'Brand - Misspell'], dtype=object)

In [126]:
models = []
for adgroup in qe_df['Ad group'].unique():
    adgroup_data = qe_df[qe_df['Ad group'] == adgroup]
    adgroup_model = {
        'name': adgroup,
        'CPC': adgroup_data['Cost'].sum()/adgroup_data['Clicks'].sum(),
        'CTR': adgroup_data['Clicks'].sum()/adgroup_data['Impressions'].sum(),
        'Conversion rate': adgroup_data['Converted clicks'].sum()/adgroup_data['Clicks'].sum()
    }
    models.append(adgroup_model)

In [127]:
models

[{'CPC': 1055333.3333333333,
  'CTR': 0.05952380952380952,
  'Conversion rate': 0.13333333333333333,
  'name': 'Brand'},
 {'CPC': 1930000.0,
  'CTR': 0.03389830508474576,
  'Conversion rate': 0.5,
  'name': 'Brand - Misspell'}]

In [128]:
b = pandas.DataFrame(models)

In [138]:
b

,CPC,CTR,Conversion rate,name
0,1055333.333333,0.059524,0.133333,Brand
1,1930000.000000,0.033898,0.500000,Brand - Misspell


In [139]:
b['CPC'][0]

1055333.3333333333